In [26]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import captcha_setting
import operator
import torchvision.transforms as transforms
from PIL import Image
import cv2 as cv
import os
from matplotlib import pyplot as plt
import numpy as np
import copy
#for eachimg in filter_containor:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),  # drop 50% of the neuron
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.Dropout(0.1),  # drop 50% of the neuron
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.Dropout(0.1),  # drop 50% of the neuron
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Sequential(
            nn.Linear((captcha_setting.IMAGE_WIDTH//8)*(captcha_setting.IMAGE_HEIGHT//8)*64, 1024),
            nn.Dropout(0.1),  # drop 50% of the neuron
            nn.ReLU())
        self.rfc = nn.Sequential(
            nn.Linear(1024, 256),#captcha_setting.MAX_CAPTCHA*captcha_setting.ALL_CHAR_SET_LEN),
            nn.ReLU()
        )
        self.rfc2 = nn.Sequential(
            nn.Linear(256, captcha_setting.MAX_CAPTCHA*captcha_setting.ALL_CHAR_SET_LEN),
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        #print(out.shape)
        out = self.rfc(out)
        out = self.rfc2(out)
        #out = out.view(out.size(0), -1)
        #print(out.shape)
        return out
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cnn = CNN()
cnn = CNN()
cnn.eval()
cnn.load_state_dict(torch.load('model_digit.pkl'))
cnn.to(device)
transform = transforms.Compose([
    # transforms.ColorJitter(),
    transforms.Grayscale(),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [25]:
root_path = '/home/ning_a/Desktop/CAPTCHA/dark_web_captcha/rescator_me/'
img_list = os.listdir(root_path)
correct = 0
total = 0
for img_cur_path in img_list:
    total += 1
    label_predicted = ''
    img = cv.imread(root_path+img_cur_path)
    img_temp = cv.imread(root_path+img_cur_path)
#     plt.imshow(img)
#     plt.show()
    n_img = np.zeros((img.shape[0],img.shape[1]))
    img_aft = cv.normalize(img, n_img, 0,255,cv.NORM_MINMAX)
#     plt.imshow(img_aft)
#     plt.show()
    gray = cv.cvtColor(img_aft,cv.COLOR_BGR2GRAY)
    ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
    ret, thresh_reverse = cv.threshold(gray,0,255,cv.THRESH_OTSU)
    #thresh_reverse = cv.cvtColor(thresh_reverse,cv.COLOR_BGR2GRAY)
#     plt.imshow(thresh)
#     plt.show()
    
    im2,contours,hierarchy = cv.findContours(thresh,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    filter_containor = []
    temp_img = copy.deepcopy(img)
    cur_contours = []
    #print(len(contours))
    for i in contours:
        #print(i)
        x, y, w, h = cv.boundingRect(i)   
        cur_contours.append([x, y, w, h])
        #break
    contours = sorted(cur_contours, key=operator.itemgetter(0))
    #print(len(contours))
    for i in range(0,len(contours)):  
        x = contours[i][0]
        y = contours[i][1]
        w = contours[i][2] 
        h = contours[i][3]
        #= cv.boundingRect(contours[i])   
        newimage=thresh_reverse[y:y+h,x:x+w] # 先用y确定高，再用x确定宽
        nrootdir=("cut_image/")
        if h<5 and w<5:
            continue
        color = [255, 255, 255]
        top, bottom, left, right = [1]*4

        newimage = cv.copyMakeBorder(newimage, top, bottom, left, right, cv.BORDER_CONSTANT, value=color)
        #newimage = 
        newimage = cv.resize(newimage,(30, 60), interpolation = cv.INTER_CUBIC)
        #filter_containor.append(newimage)

        cv.rectangle(temp_img, (x,y), (x+w,y+h), (153,153,0), 1)
        if not os.path.isdir(nrootdir):
            os.makedirs(nrootdir)
        cv.imwrite( nrootdir+str(i)+".jpg",newimage)
        cv.imwrite( "temp.jpg",newimage) 
        filter_containor.append(Image.open("temp.jpg"))
    filter_containor = []
    for i in range(5):
#         print(img_temp.shape)
        cv.imwrite( "temp2.jpg",img_temp[:,i*32:i*32+32])
        filter_containor.append(Image.open("temp.jpg"))
    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    for eachimg in filter_containor:
        #print(eachimg)
        image = transform(eachimg).unsqueeze(0)
#         plt.imshow(eachimg)
#         plt.show()
#         print(image.shape)
        image = torch.tensor(image, device=device).float()
        image = Variable(image).to(device)
        #print(image.shape)
        #image, labels =  image.to(device), labels.to(device)
        # vimage = generator(image)
        predict_label = cnn(image)
        #labels = labels.cpu()
        predict_label = predict_label.cpu()
        _, predicted = torch.max(predict_label, 1)
        label_predicted+= captcha_setting.ALL_CHAR_SET[predicted]
#         print(captcha_setting.ALL_CHAR_SET[predicted])
    parsed_label = ''
    if('_' in img_cur_path):
        parsed_label = img_cur_path.split('_')[0]
    else:
        parsed_label = img_cur_path.split('.')[0]
    if(parsed_label == label_predicted):
        correct +=1
#     print(label_predicted)
#     print(parsed_label)
    
#     break
print(correct/total)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0.0
